In [1]:
import time
import numpy as np
import pandas as pd

from pprint import pprint
from datetime import datetime, date, timedelta

# Spotify Specific Libraries
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Import API key
from config import POSTGRES_USERNAME, POSTGRES_PASSWORD, POSTGRES_ADDRESS, POSTGRES_PORT, POSTGRES_DBNAME, SPOTIFY_CLIENT_ID, SPOTIFY_CLIENT_SECRET, SPOTIFY_REDIRECT_URI

In [ ]:
print(f"client_id = {SPOTIFY_CLIENT_ID} || secret = {SPOTIFY_CLIENT_SECRET}")

In [2]:
from sqlalchemy import create_engine

# A long string that contains the necessary Postgres login information
postgres_str = f"postgresql://{POSTGRES_USERNAME}:{POSTGRES_PASSWORD}@{POSTGRES_ADDRESS}:{POSTGRES_PORT}/{POSTGRES_DBNAME}"

# Create the connection
cnx = create_engine(postgres_str)

In [3]:
fileName = "D:\BootCamp\Group_Projects\Final_Project\data\songs_with_features.csv"

df_spotify_songs = pd.read_csv(fileName)
df_spotify_songs.head()

,song_title,song_id,song_artist,genre,playlist_id,playlist_title,acousticness,analysis_url,danceability,duration_ms,...,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
0,You and Me,00bq71I9OZ87M6jZqQFV6I,The Magician,edm_dance,37i9dQZF1DX4dyzvuaRJ0n,mint,0.000415,https://api.spotify.com/v1/audio-analysis/00bq...,0.646,214959,...,0.0882,-5.263,0,0.0437,121.043,4,https://api.spotify.com/v1/tracks/00bq71I9OZ87...,audio_features,spotify:track:00bq71I9OZ87M6jZqQFV6I,0.1170
1,Two Lovers,00CmjeeHvAVKvx3tcIiZTy,Mary Wells,soul,37i9dQZF1DWVGLWeeMNGcG,Evolution Of Soul,0.426000,https://api.spotify.com/v1/audio-analysis/00Cm...,0.678,166733,...,0.1090,-16.965,1,0.0304,105.902,4,https://api.spotify.com/v1/tracks/00CmjeeHvAVK...,audio_features,spotify:track:00CmjeeHvAVKvx3tcIiZTy,0.9600
2,"Lieder ohne Worte (Songs without Words), Book ...",00CXUMREit80f2McJsjcIz,Felix Mendelssohn,classical,37i9dQZF1DWUvHZA1zLcjW,Chilled Classical,0.995000,https://api.spotify.com/v1/audio-analysis/00CX...,0.355,282813,...,0.0986,-28.659,1,0.0396,67.946,4,https://api.spotify.com/v1/tracks/00CXUMREit80...,audio_features,spotify:track:00CXUMREit80f2McJsjcIz,0.0922
3,Cita,00dlNWEiPTtm1ldy6zOUlz,Prince Royce,latin,37i9dQZF1DWT34oeYRnJ0R,Vibras,0.079400,https://api.spotify.com/v1/audio-analysis/00dl...,0.731,216347,...,0.6210,-6.606,0,0.0392,122.034,4,https://api.spotify.com/v1/tracks/00dlNWEiPTtm...,audio_features,spotify:track:00dlNWEiPTtm1ldy6zOUlz,0.6620
4,What’s Done Is Done,00EFWaqXnHZ5smJNsHtnGV,Seven Lions,edm_dance,37i9dQZF1DX8tZsk68tuDw,Dance Rising,0.422000,https://api.spotify.com/v1/audio-analysis/00EF...,0.394,290824,...,0.0980,-6.149,1,0.0451,170.017,4,https://api.spotify.com/v1/tracks/00EFWaqXnHZ5...,audio_features,spotify:track:00EFWaqXnHZ5smJNsHtnGV,0.1930


In [5]:
df_spotify_songs.to_sql('spotify_songs', cnx)

# TEST ONLY

### Proof of Concept: Read Data From DB

In [6]:
try:
    myTestdf = pd.read_sql_query("SELECT * FROM spotify_songs LIMIT 5", cnx)
    
except Error as e:
    print(f"<message>./n/n  Error is:  {e}")
finally:
    print("Done.")
# end try

Done.


In [7]:
myTestdf.head()

,index,song_title,song_id,song_artist,genre,playlist_id,playlist_title,acousticness,analysis_url,danceability,...,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
0,0,You and Me,00bq71I9OZ87M6jZqQFV6I,The Magician,edm_dance,37i9dQZF1DX4dyzvuaRJ0n,mint,0.000415,https://api.spotify.com/v1/audio-analysis/00bq...,0.646,...,0.0882,-5.263,0,0.0437,121.043,4,https://api.spotify.com/v1/tracks/00bq71I9OZ87...,audio_features,spotify:track:00bq71I9OZ87M6jZqQFV6I,0.1170
1,1,Two Lovers,00CmjeeHvAVKvx3tcIiZTy,Mary Wells,soul,37i9dQZF1DWVGLWeeMNGcG,Evolution Of Soul,0.426000,https://api.spotify.com/v1/audio-analysis/00Cm...,0.678,...,0.1090,-16.965,1,0.0304,105.902,4,https://api.spotify.com/v1/tracks/00CmjeeHvAVK...,audio_features,spotify:track:00CmjeeHvAVKvx3tcIiZTy,0.9600
2,2,"Lieder ohne Worte (Songs without Words), Book ...",00CXUMREit80f2McJsjcIz,Felix Mendelssohn,classical,37i9dQZF1DWUvHZA1zLcjW,Chilled Classical,0.995000,https://api.spotify.com/v1/audio-analysis/00CX...,0.355,...,0.0986,-28.659,1,0.0396,67.946,4,https://api.spotify.com/v1/tracks/00CXUMREit80...,audio_features,spotify:track:00CXUMREit80f2McJsjcIz,0.0922
3,3,Cita,00dlNWEiPTtm1ldy6zOUlz,Prince Royce,latin,37i9dQZF1DWT34oeYRnJ0R,Vibras,0.079400,https://api.spotify.com/v1/audio-analysis/00dl...,0.731,...,0.6210,-6.606,0,0.0392,122.034,4,https://api.spotify.com/v1/tracks/00dlNWEiPTtm...,audio_features,spotify:track:00dlNWEiPTtm1ldy6zOUlz,0.6620
4,4,What’s Done Is Done,00EFWaqXnHZ5smJNsHtnGV,Seven Lions,edm_dance,37i9dQZF1DX8tZsk68tuDw,Dance Rising,0.422000,https://api.spotify.com/v1/audio-analysis/00EF...,0.394,...,0.0980,-6.149,1,0.0451,170.017,4,https://api.spotify.com/v1/tracks/00EFWaqXnHZ5...,audio_features,spotify:track:00EFWaqXnHZ5smJNsHtnGV,0.1930


# TEST - Spotify API

### Proof of Concept: Pull Data From Each Spotify API We Will Be Using

In [ ]:
# on Mac, use `export` instead of `set`
# set SPOTIPY_CLIENT_ID = SPOTIFY_CLIENT_ID
# set SPOTIPY_CLIENT_SECRET = SPOTIFY_CLIENT_SECRET

baseURL = "https://api.spotify.com"

cid = SPOTIFY_CLIENT_ID
secret = SPOTIFY_CLIENT_SECRET

# Next, create a Spotify object and call methods:
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [ ]:
results = sp.search(q='stick figure', limit=1)
print(results)
# for idx, track in enumerate(results['tracks']['items']):
#     print(idx, track['name'])

In [ ]:
results = sp.category_playlists(category_id="party", country="US", limit=20, offset=0)
# for idx, track in enumerate(results['tracks']['items']):
#     print(idx, track['name'])
print(results)

In [ ]:
print(results)

In [ ]:
# audio_analysis(track_id)
# Get audio analysis for a track based upon its Spotify ID Parameters:

# track_id - a track URI, URL or ID
# audio_features(tracks=[])
# Get audio features for one or multiple tracks based upon their Spotify IDs Parameters:

# tracks - a list of track URIs, URLs or IDs, maximum: 100 ids

In [ ]:
trackID = 'spotify:track:3W4x7fZhMHw2pE6wyBzzF8'

trackResults = sp.audio_analysis(trackID)
print(trackResults)

In [ ]:
trackResults = sp.audio_features(tracks = trackID)
print(trackResults)

In [ ]:
trackResults[0]["energy"]